In [1]:
!pip install langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.2 MB/s eta 0:00:00


In [4]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

# --- 1) 分類したいラベルを定義（ここを差し替え） ---
LABELS = ["有害", "迷惑", "一般"]

# --- 2) JSON Schema（ラベルは enum で厳密指定）---
schema = {
    "title": "Classification",
    "type": "object",
    "properties": {
        "label": {
            "type": "string",
            "enum": LABELS,
            "description": "必ず上記のいずれか1つを選ぶこと"
        },
        "confidence": {
            "type": "number",
            "minimum": 0.0,
            "maximum": 1.0,
            "description": "判断の確信度（0〜1）"
        },
        "reason": {
            "type": "string",
            "description": "そのラベルにした短い根拠"
        }
    },
    "required": ["label", "confidence", "reason"],
    "additionalProperties": False
}

# --- 3) プロンプト（LCEL）---
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "あなたは厳密なテキスト分類器です。次の候補から必ず1つだけ選んでください: {labels}\n"
     "出力はスキーマに厳密に従ってください。余計な説明は出力しないでください。"),
    ("human", "テキスト:\n{text}")
])

# --- 4) モデル（OpenAI）。環境変数 OPENAI_API_KEY を利用 ---
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0,
                 api_key = api_key)

# --- 5) チェーン構築（構造化出力）---
chain = prompt | llm.with_structured_output(schema)



In [6]:
# --- 6) 単発実行の例 ---
text = "殺すぞ、と脅されました。"
result = chain.invoke({"labels": ", ".join(LABELS), "text": text})
# result は dict: {"label": "...", "confidence": 0.x, "reason": "..."}
print(result)

# --- 7) バッチ分類の例 ---
texts = [
    "今だけ無料のiPhoneを当選しました！こちらをクリック",
    "今日の会議は15時からです。",
    "住所やマイナンバーを送ってください。"
]
batched_inputs = [{"labels": ", ".join(LABELS), "text": t} for t in texts]
results = chain.batch(batched_inputs, config={"max_concurrency": 4})
for t, r in zip(texts, results):
    print("----")
    print("TEXT:", t)
    print("OUT :", r)

{'label': '\x00\x00', 'confidence': 0.9, 'reason': '脅迫的な表現が含まれており、有害な内容と判断されるため。'}
----
TEXT: 今だけ無料のiPhoneを当選しました！こちらをクリック
OUT : {'label': '\x00\x00', 'confidence': 0.9, 'reason': '無料のiPhoneを当選したという内容は、詐欺や迷惑な勧誘の可能性が高いと判断されるため。'}
----
TEXT: 今日の会議は15時からです。
OUT : {'label': '\x00\x00', 'confidence': 0.9, 'reason': '一般的な情報提供であり、有害または迷惑ではないため。'}
----
TEXT: 住所やマイナンバーを送ってください。
OUT : {'label': '\x00\x00', 'confidence': 0.9, 'reason': '個人情報の提供を求める内容であり、プライバシーに関わる有害なリクエストと判断されるため。'}
